In [2]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import cv2
from PIL import Image, ImageDraw
from IPython import display

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [4]:
mtcnn = MTCNN(keep_all=True, device=device,select_largest=False)

In [7]:
def mosaic(src, ratio=0.1):
    small = cv2.resize(src, None, fx=ratio, fy=ratio, interpolation=cv2.INTER_NEAREST)
    return cv2.resize(small, src.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)

In [8]:
def mosaic_area(src, x, y, width, height, ratio=0.1):
    dst = src.copy()
    dst[y:y + height, x:x + width] = mosaic(dst[y:y + height, x:x + width], ratio)
    return dst

In [9]:
webcam = cv2.VideoCapture(0)


w = round(webcam.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = webcam.get(cv2.CAP_PROP_FPS) # 카메라에 따라 값이 정상적, 비정상적

fourcc = cv2.VideoWriter_fourcc(*'DIVX')

out = cv2.VideoWriter('output.avi', fourcc, fps, (w, h))

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

while webcam.isOpened():

    status, frame = webcam.read()

    boxes,prob=mtcnn.detect(frame,landmarks=False)



    
    #frame_draw =frame.copy()
    if boxes is None:
        print("얼굴 인식에 실패하였습니다")
        continue
    boxes=boxes.astype(int)
    max_box=prob.argmax()
    #crop_frame=frame_draw[boxes[max_box][1]:boxes[max_box][3],boxes[max_box][0]:boxes[max_box][2]]
    frame=mosaic_area(frame,boxes[max_box][0],boxes[max_box][1],boxes[max_box][3]-boxes[max_box][1],boxes[max_box][2]-boxes[max_box][0])
    

    
    # 웹캠으로 찰영한 영상을 저장하기
    # cv2.VideoWriter 객체 생성, 기존에 받아온 속성값 입력
    if status:

        out.write(frame)

        cv2.imshow("test", frame)
        
#        cv2.imshow('crop_face',crop_frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



webcam.release()
out.release()
cv2.destroyAllWindows()

얼굴 인식에 실패하였습니다
얼굴 인식에 실패하였습니다


In [6]:
webcam = cv2.VideoCapture(0)


if not webcam.isOpened():
    print("Could not open webcam")
    exit()

while webcam.isOpened():
    status, frame = webcam.read()

    boxes,prob,points=mtcnn.detect(frame,landmarks=True)
    
    frame_draw =frame.copy()
    
    if boxes is None:
        print("얼굴 인식에 실패하였습니다")
        continue
        
    boxes=boxes.astype(int)
    points=points.astype(int)

    max_box=prob.argmax()
    crop_frame=frame_draw[boxes[max_box][1]:boxes[max_box][3],boxes[max_box][0]:boxes[max_box][2]]
    
    ly = (points[0][3][1]+points[0][3][1])//2
    y = boxes[max_box][3]-boxes[max_box][1]
    lip = frame_draw[ly-y//8:ly+y//8, points[0][3][0] : points[0][4][0]]

    cv2.circle(frame, (points[0][3][0], points[0][3][1]), 5, (0, 0, 255), -1)

    

    
    if status:
        
        cv2.imshow("test", frame)
        cv2.imshow('crop_face',crop_frame)
        
        cv2.imshow('lip',lip)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 